In [1]:
#If you have trouble using Gurobi
using Pkg
Pkg.rm("Gurobi")
Pkg.add("Gurobi")
Pkg.add("CSV")
Pkg.add("DataFrames")

    Updating `C:\Users\Admin\.julia\environments\v1.10\Project.toml`
  [2e9cd046] - Gurobi v1.3.0
    Updating `C:\Users\Admin\.julia\environments\v1.10\Manifest.toml`
  [2e9cd046] - Gurobi v1.3.0
  [c018c7e6] - Gurobi_jll v11.0.2+2
        Info We haven't cleaned this depot up for a bit, running Pkg.gc()...
      Active manifest files: 1 found
      Active artifact files: 85 found
      Active scratchspaces: 9 found
     Deleted 3 package installations (188.231 MiB)
    Updating registry at `C:\Users\Admin\.julia\registries\General.toml`
   Resolving package versions...
    Updating `C:\Users\Admin\.julia\environments\v1.10\Project.toml`
  [2e9cd046] + Gurobi v1.3.0
    Updating `C:\Users\Admin\.julia\environments\v1.10\Manifest.toml`
  [2e9cd046] + Gurobi v1.3.0
  [c018c7e6] + Gurobi_jll v11.0.2+2
   Resolving package versions...
  No Changes to `C:\Users\Admin\.julia\environments\v1.10\Project.toml`
  No Changes to `C:\Users\Admin\.julia\environments\v1.10\Manifest.toml`
   Resolvin

In [2]:
using Gurobi, JuMP, Distributions

In [2]:
using JuMP
using Gurobi
using Distributions
using CSV
using DataFrames

# Load the CSV file into DataFrames
demand_data = CSV.read(raw"C:\Users\Admin\Downloads\demand_data8.csv", DataFrame)
para_data = CSV.read(raw"C:\Users\Admin\Downloads\data8.csv", DataFrame)

# Extract the relevant data from the parameter DataFrame
ci = para_data.ci[1]
cr = para_data.cr[1]
co = para_data.co[1]
cu = para_data.cu[1]
cs = para_data.cs[1]
cb = para_data.cb[1]
W_init = para_data.W_init[1]
I_init = para_data.I_init[1]
K = para_data.K[1]
T = para_data.T[1]
B_init = para_data.B_init[1]

# Verify the dimensions of the demand_data DataFrame
n_periods = size(demand_data, 1)  # Number of periods
n_products = size(demand_data, 2) - 1  # Assuming the first column is 'Product'
D = Array{Float64}(undef, n_periods, n_products)

# Transpose the demand data to D[t, i]
for t in 1:n_periods
    for i in 1:n_products
        D[t, i] = demand_data[t, i + 1]  # Adjust for 1-based indexing in Julia
    end
end

# Create the model with Gurobi optimizer
m = Model(Gurobi.Optimizer)
n = T
nt = 20
k_max = 5
j_max = 5

# Define the variables
@variable(m, W[0:n, 1:k_max, 1:j_max] >= 0, Int)
@variable(m, H[1:n, 1:k_max, 1:j_max] >= 0, Int)
@variable(m, F[1:n, 1:k_max, 1:j_max] >= 0, Int)
@variable(m, I[0:n, 1:k_max, 1:j_max] >= 0)
@variable(m, P[1:n, 1:k_max, 1:j_max] >= 0)
@variable(m, O[1:n, 1:k_max, 1:j_max] >= 0)
@variable(m, U[1:n, 1:k_max, 1:j_max] >= 0)
@variable(m, S[1:n, 1:k_max, 1:j_max] >= 0, Int)
@variable(m, B[0:n, 1:k_max, 1:j_max] >= 0, Int)

# Define constants
cf = 80
ch = 70

# Add initial constraints for W[0] and I[0]
for k in 1:k_max
    for j in 1:j_max
        @constraint(m, W[0, k, j] == W_init)
        @constraint(m, I[0, k, j] == I_init)
        @constraint(m, B[0, k, j] == B_init)
    end
end

# Define the constraints
for i in 1:n_periods
    for k in 1:k_max
        for j in 1:j_max
            @constraint(m, W[i, k, j] == W[i-1, k, j] + H[i, k, j] - F[i, k, j])
            @constraint(m, P[i, k, j] == K * nt * W[i, k, j] + O[i, k, j] - U[i, k, j])
            @constraint(m, I[i, k, j] - B[i, k, j] == I[i-1, k, j] - B[i-1, k, j] + P[i, k, j] + S[i, k, j] - D[i, 1])  # Ensure D[i, 1] is correctly indexed
            @constraint(m, H[i, k, j] <= 0.1 * W[i, k, j])
            @constraint(m, F[i, k, j] <= 0.1 * W[i, k, j])
        end
    end
end

# Define the objective function
@objective(m, Min,
    ch * sum(H[i, k, j] for i in 1:n, k in 1:k_max, j in 1:j_max) +
    cf * sum(F[i, k, j] for i in 1:n, k in 1:k_max, j in 1:j_max) +
    ci * sum(I[i, k, j] for i in 1:n, k in 1:k_max, j in 1:j_max) +
    co * sum(O[i, k, j] for i in 1:n, k in 1:k_max, j in 1:j_max) +
    cu * sum(U[i, k, j] for i in 1:n, k in 1:k_max, j in 1:j_max) +
    cs * sum(S[i, k, j] for i in 1:n, k in 1:k_max, j in 1:j_max) +
    cr * sum(P[i, k, j] for i in 1:n, k in 1:k_max, j in 1:j_max) +
    cb * sum(B[i, k, j] for i in 1:n, k in 1:k_max, j in 1:j_max)
)

# Optimize the model
optimize!(m)

# Collect results into a DataFrame
results = DataFrame(
    i = Int[],
    k = Int[],
    j = Int[],
    W = Float64[],
    H = Float64[],
    F = Float64[],
    I = Float64[],
    P = Float64[],
    O = Float64[],
    U = Float64[],
    S = Float64[],
    B = Float64[]
)

for i in 1:n_periods
    for k in 1:k_max
        for j in 1:j_max
            push!(results, (
                i, k, j,
                value(W[i, k, j]),
                value(H[i, k, j]),
                value(F[i, k, j]),
                value(I[i, k, j]),
                value(P[i, k, j]),
                value(O[i, k, j]),
                value(U[i, k, j]),
                value(S[i, k, j]),
                value(B[i, k, j])
            ))
        end
    end
end

# Save the results to a CSV file
CSV.write(raw"C:\Users\Admin\Downloads\results.csv", results)
# Print the results
for i in 1:n_periods
    for k in 1:k_max
        for j in 1:j_max
            println("W[$i, $k, $j] = ", value(W[i, k, j]))
            println("H[$i, $k, $j] = ", value(H[i, k, j]))
            println("F[$i, $k, $j] = ", value(F[i, k, j]))
            println("I[$i, $k, $j] = ", value(I[i, k, j]))
            println("P[$i, $k, $j] = ", value(P[i, k, j]))
            println("O[$i, $k, $j] = ", value(O[i, k, j]))
            println("U[$i, $k, $j] = ", value(U[i, k, j]))
            println("S[$i, $k, $j] = ", value(S[i, k, j]))
            println("B[$i, $k, $j] = ", value(B[i, k, j]))
        end
    end
end


Set parameter Username
Academic license - for non-commercial use only - expires 2025-04-18
Gurobi Optimizer version 11.0.2 build v11.0.2rc0 (win64 - Windows 11.0 (22631.2))

CPU model: Intel(R) Core(TM) i5-8265U CPU @ 1.60GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1575 rows, 8175 columns and 5475 nonzeros
Model fingerprint: 0x63277e6d
Variable types: 3625 continuous, 4550 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e-01, 3e+00]
  Objective range  [4e+00, 1e+02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+03]
Presolve removed 1515 rows and 8081 columns
Presolve time: 0.00s
Presolved: 60 rows, 94 columns, 199 nonzeros
Variable types: 35 continuous, 59 integer (0 binary)
Found heuristic solution: objective 2.317381e+07

Root relaxation: objective 1.291956e+07, 51 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bo

In [54]:
demand_data = CSV.read(raw"C:\Users\Admin\Downloads\demand_data8.csv", DataFrame)
para_data = CSV.read(raw"C:\Users\Admin\Downloads\data8.csv", DataFrame)
workers_data = CSV.read(raw"C:\Users\Admin\Downloads\worker_data.csv", DataFrame)
demand_data

Row,Column1,x1,x2,x3,x4,x5,x6,x7,x8,x9,x10
,String7,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,I_init,434,0,534,1022,0,0,849,693,0,1209
2,B_init,0,344,0,0,399,244,0,0,1044,0
3,t1,598,952,1273,780,627,289,311,1856,1594,1
4,t2,1690,502,181,1662,287,1574,953,1002,1759,1532
5,t3,820,1819,1865,1680,420,1503,574,299,82,1391
6,t4,1956,714,1295,434,1241,1925,1641,917,1479,1861
7,t5,1039,1131,8,512,701,1825,489,198,958,710
8,t6,186,1109,1545,1859,288,23,941,1042,90,155
9,t7,1195,235,372,280,1987,1013,581,577,1413,1451


In [57]:
K

3×10 Matrix{Float64}:
 300.0  299.0  304.0  279.0  296.0  300.0  299.0  304.0  300.0  279.0
 282.0  250.0  294.0  300.0  287.0  282.0  250.0  294.0  282.0  300.0
 302.0  276.0  301.0  301.0  300.0  302.0  276.0  301.0  302.0  301.0

In [64]:
demand_data[2, 2]

0

In [1]:
using JuMP
using Gurobi
using Distributions
using CSV
using DataFrames

export some_function, run_optimizer, printhi

# Unhash these to test-run the LP directly from here
#demand_file_path = joinpath("uploads", "demand", "demand_data9.csv")
#para_file_path = joinpath("uploads", "parameters", "data8.csv")
#workers_file_path = joinpath("uploads", "workers", "worker_data.csv")
#demand_data = CSV.read(demand_file_path, DataFrame)
#para_data = CSV.read(para_file_path, DataFrame)
#workers_data = CSV.read(workers_file_path, DataFrame)
demand_data = CSV.read(raw"C:\Users\Admin\Downloads\demand_data8.csv", DataFrame)
para_data = CSV.read(raw"C:\Users\Admin\Downloads\data8.csv", DataFrame)
workers_data = CSV.read(raw"C:\Users\Admin\Downloads\worker_data.csv", DataFrame)

#Run the optimizer
function run_optimizer(demand_data, para_data, workers_data, AC_List)

    @info "Checkpoint: Function accessed"

    # Extract the relevant data from the para_data DataFrame
    
    ch = para_data.ch[1]
    cf = para_data.cf[1]
    ci = para_data.ci[1]
    cr = para_data.cr[1]
    co = para_data.co[1]
    cu = para_data.cu[1]
    cs = para_data.cs[1]
    cb = para_data.cb[1]

    # Verify the dimensions of the demand_data DataFrame
    n_periods = size(demand_data, 1) - 2  # Number of periods
    n_products = size(demand_data, 2) - 1 # Assuming the first row is header with product index i
    D = Array{Float64}(undef, n_periods, n_products)

    D = Array{Float64}(undef, n_periods, n_products)
    for t in 2:n_periods + 1
        for i in 2:n_products + 1
            D[t - 1, i - 1] = demand_data[t + 1, i]  # Adjust for 1-based indexing in Julia
        end
    end
    # Extract K (Worker production rate) array
    n_workers = size(workers_data, 1)   #number of workers

    # Test that number of products matches for Demand and Workers data
    if n_products == size(workers_data, 2) - 2
        nothing 
    else
        @error "Error: Number of product types in demand CSV and workers CSV do not match. Please check your uploads."
    end

    K = Array{Float64}(undef, n_workers,n_products)
    for i in 3:n_products + 2
        for j in 1:n_workers
            K[j, i - 2] = workers_data[j, i]  # Adjust for 1-based indexing in Julia
        end
    end
    @info "Checkpoint: Data Extracted"

    # Create the model with Gurobi optimizer
    m = Model(Gurobi.Optimizer)
    i_max = n_products #product type
    j_max = n_workers #worker type

    @info "Checkpoint: Model Initialized"

    # Define the variables
    @variable(m, W[0:n_periods, 1:i_max, 1:j_max] >= 0, Int)
    @variable(m, Wi[0:n_periods, 1:j_max] >= 0, Int)    
    @variable(m, H[1:n_periods, 1:j_max] >= 0, Int)
    @variable(m, F[1:n_periods, 1:j_max] >= 0, Int)
    @variable(m, I[0:n_periods, 1:i_max] >= 0)
    @variable(m, P[1:n_periods, 1:i_max] >= 0)
    @variable(m, O[1:n_periods, 1:i_max] >= 0)
    @variable(m, U[1:n_periods, 1:i_max] >= 0)
    @variable(m, S[1:n_periods, 1:i_max] >= 0, Int)
    @variable(m, B[0:n_periods, 1:i_max] >= 0, Int)
    @variable(m, R[0:n_periods, 1:i_max] >= 0, )

    @info "Checkpoint: Defined Variables"
    
    # Add initial constraints for W[0] and I[0]  

    for j in 1:j_max
        @constraint(m, Wi[0, j] == workers_data[j, 2])
    end
    
    for i in 1:i_max
        @constraint(m, I[0, i] == demand_data[1, i+1])
        @constraint(m, B[0, i] == demand_data[2, i+1])
    end

    # Define the constraints

    # Workforce constraints
    for j in 1:j_max
        for t in 0:n_periods
            @constraint(m, Wi[t, j] == sum(W[t,i,j] for i in 1:i_max))
        end
    end

    for t in 1:n_periods
         for j in 1:j_max
            @constraint(m, Wi[t, j] == Wi[t-1, j] + H[t, j] - F[t, j])
            @constraint(m, H[t, j] <= 0.1 * Wi[t,j])
            @constraint(m, F[t, j] <= 0.1 * Wi[t,j])
        end
    end

    # Production Constraints
    for t in 1:n_periods
        for i in 1:i_max
            for j in 1:j_max
                @constraint(m, P[t, i] == R[t, i] + O[t, i] - U[t, i])
                @constraint(m, R[t,i] <= (K[j,i]*W[t,i,j]))
                @constraint(m, O[t,i] <= 0.25 * R[t,i])
                
            end
        end
    end

    # Demand Constraints
    for t in 1:n_periods
        for i in 1:i_max
            for j in 1:j_max
                @constraint(m, I[t, i] - B[t, i] == I[t-1, i] - B[t-1, i] + P[t, i] + S[t, i] - D[t,i])  # Ensure D[i, 1] is correctly indexed
                @constraint(m, B[t,i] <= 0.25 * P[t,i])
            end
        end
    end
    @info "Checkpoint: Defined constraints"

    #Additional constraints
    if !isempty(AC_List)
        try
            for item in AC_List #Each item inside AC_List is itself a list. 1 item defines 1 single constraint.

                #[Index 1: Constraint applies to Hiring, Firing, or Inventory?
                # Index 2: t (production period i.e. month week etc.)
                # Index 3: i (product type)
                # Index 4: j (worker type)
                # Index 5: H/F/I is <=, =, or >= some value?
                # Index 6: Value]
                @info item
                if item[1] == "H" #Index 1
                    if item[5] == "<="
                        @constraint(m, H[item[2], item[4]] <= item[6])
                    elseif item[5] == "="
                        @constraint(m, H[item[2], item[4]] == item[6])
                    elseif item[5] == ">="
                        @constraint(m, H[item[2], item[4]] >= item[6])
                    end
                end
    
                if item[1] == "F"
                    if item[5] == "<="
                        @constraint(m, F[item[2], item[4]] <= item[6])
                    elseif item[5] == "="
                        @constraint(m, F[item[2], item[4]] == item[6])
                    elseif item[5] == ">="
                        @constraint(m, F[item[2], item[4]] >= item[6])
                    end
                end
    
                if item[1] == "I"
                    if item[5] == "<="
                        @constraint(m, F[item[2], item[3]] <= item[6])                        
                    elseif item[5] == "="
                        @constraint(m, F[item[2], item[3]] == item[6])
                    elseif item[5] == ">="
                        @constraint(m, F[item[2], item[3]] >= item[6])
                    end
                end
            end
            @info "Checkpoint: Additional Constraints received successfully."
                
        catch e
            @error "Error: Additional Constraints defined incorrectly. Please check Additional Constraint inputs."
        end

    end
    

    # Define the objective function
    @objective(m, Min,
    ch * sum(H[t,j] for t in 1:n_periods, j in 1:j_max) +
    cf * sum(F[t,j] for t in 1:n_periods, j in 1:j_max) +
    ci * sum(I[t,i] for t in 1:n_periods, i in 1:i_max) +
    co * sum(O[t,i] for t in 1:n_periods, i in 1:i_max) +
    cu * sum(U[t,i] for t in 1:n_periods, i in 1:i_max) +
    cs * sum(S[t,i] for  t in 1:n_periods, i in 1:i_max) +
    cr * sum(P[t,i] for  t in 1:n_periods, i in 1:i_max) +
    cb * sum(B[t,i] for t in 1:n_periods, i in 1:i_max)
    )

    @info "Checkpoint: Defined Objective Function"

    # Optimize the model
    optimize!(m)

    @info "Checkpoint: Optimized"

    # Collect results into a DataFrame
    results = DataFrame(
        t = Int[],
        i = Int[],
        j = Int[],
        W = Float64[],
        H = Float64[],
        F = Float64[],
        I = Float64[],
        P = Float64[],
        O = Float64[],
        U = Float64[],
        S = Float64[],
        B = Float64[]
    )

    for t in 1:n_periods
        for i in 1:i_max
            for j in 1:j_max
                push!(results, (
                    t, i, j,
                    value(W[t, i, j]),
                    value(H[t, j]),
                    value(F[t, j]),
                    value(I[t, i]),
                    value(P[t, i]),
                    value(O[t, i]),
                    value(U[t, i]),
                    value(S[t, i]),
                    value(B[t, i])
                ))
            end
        end
    end

    @info "Checkpoint: results saved"

    # Save the results to a CSV file
    result_path = joinpath("lib", "downloads", "workforce_inventory.csv")
    @info result_path
    try
        CSV.write(result_path, results)
    catch e
        @error e
    end

    @info "Checkpoint: results written to file"

    # Print the results
#    for t in 1:n_periods
#        for i in 1:i_max
#            for j in 1:j_max
#                println("W[$t, $i, $j] = ", value(W[t, i, j]))
#                println("H[$t, $j] = ", value(H[t, j]))
#                println("F[$t, $j] = ", value(F[t, j]))
#                println("I[$t, $i] = ", value(I[t, i]))
#                println("P[$t, $i] = ", value(P[t, i]))
#                println("O[$t, $i] = ", value(O[t, i]))
#                println("U[$t, $i] = ", value(U[t, i]))
#                println("S[$t, $i] = ", value(S[t, i]))
#                println("B[$t, $i] = ", value(B[t, i]))
#            end
#        end
#    end
    @info "Checkpoint: results printed (disabled)"
    @info "Success!"

    return results, objective_value(m)
end

#Dummy functions for testing
#We can delete this at the end but keep them here for now to help us
function printhi()
    @info "Hi!! I'm working :D"
end

function some_function(K)
    K = push!(K, 2)
    return K
end
end

LoadError: ParseError:
[90m# Error @ [0;0m]8;;file://c:/Users/Admin/OneDrive - Singapore University of Technology and Design/Term 5/MSO Project/Term-5-2D-Project/Backend/In[1]#286:1\[90mIn[1]:286:1[0;0m]8;;\
end
[48;2;120;70;70mend[0;0m
[90m└─┘ ── [0;0m[91minvalid identifier[0;0m

In [2]:
ci,cr,co,cu,cs,W_init,I_init,K


(59702.808482688655, 40277.96711584799, 32264.816291429455, 96690.61531409521, 11111.560979431235, 94175, 92924, 16.914360023083187)

In [26]:
cb,B_init

(13060.126187629465, 1548)

In [38]:
println("The type of ci is: ", typeof(ci))

The type of ci is: Float64


In [39]:
print(m)